In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED=42

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')
sample = pd.read_csv(f'{data_path}sample_submission.csv')
train_gpt = pd.read_csv(f'{data_path}train_gpt.csv')


train_ft = pd.read_csv(data_path + 'train_80000_rule_based.csv')


In [ ]:
train_ft

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
79995,마감하자 + 인테리어,"도배지에 울음이 생겼어 또한, 2023년의 최신 인테리어 디자인 트렌드가 무엇인가요?",도배지 울음이란 도배지가 평평하지 않고 국소적으로 볼록하게 튀어나오는 현상을 의미합...
79996,마감재 + 시공,"그라스울의 단점이 뭐야? 또한, 액체방수공사가 뭐야?",그라스울의 단점은 유리 조직으로써 맨손으로 만지면 다칠 수 있다는 점과 내부마감을 ...
79997,마감재 + 마감하자,포세린 타일이 뭐야? 그리고 부적절한 접착제를 사용하면 도배지꼬임이 발생할 수 있어?,포세린 타일은 점토를 주원료로 만들어진 무광의 타일을 말합니다. 이는 여름에는 시원...
79998,마감하자 + 타 마감하자,"구조적 결함으로 인해 석고수정이 발생할 수 있어? 또한, 결로 판정 기준이 어떻게 돼?","건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 또한, 결로 판정 ..."


In [ ]:
import os, torch, tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!nvidia-smi

Fri Mar  1 23:55:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# 1. megastudyedu/M-SOLAR-10.7B-v1.3

## 1 모델 Peft

In [ ]:
import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
# from trl import SFTTrainer

In [ ]:
model_ID = "megastudyedu/M-SOLAR-10.7B-v1.3"

# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    use_nested_quant = False
)

tokenizer = AutoTokenizer.from_pretrained(model_ID)
model = AutoModelForCausalLM.from_pretrained(
    model_ID,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer


In [ ]:
user_request = "실내장식이 뭐야?"

conversation = [ {'role': 'user', 'content': user_request} ]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])ㅌ
output_text = output_text.split("### Assistant:")[1].strip()

print(output_text)

In [ ]:
user_request = "실내장식이 뭐야?"

conversation = [ {'role': 'user', 'content': user_request} ]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, use_cache=True, max_length=200)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
실내장식이 뭐야?

### Assistant:
 실내장식은 실내 공간을 개조하고 장식하는 예술과 기술입니다. 실내장식가는 실내 공간의 기능, 미학, 안전을 고려하여 실내 공간을 설계하고 장식합니다. 실내장식은 건축, 디자인, 공학, 조경, 조명, 건축 기술 등 다양한 분야의 전문 지식을 결합한 것입니다. 실내장식가는 건물 설계, 공간 계획, 


In [ ]:
assistant_response = output_text.split("### Assistant:")[1].strip()
print(assistant_response)

실내장식은 실내 공간을 개조하고 장식하는 예술과 기술입니다. 실내장식가는 실내 공간의 기능, 미학, 안전을 고려하여 실내 공간을 설계하고 장식합니다. 실내장식은 건축, 디자인, 공학, 조경, 조명, 건축 기술 등 다양한 분야의 전문 지식을 결합한 것입니다. 실내장식가는 건물 설계, 공간 계획,


In [ ]:
# prompt = '### User:\n 실내 장식이 뭐야? \n\n### Assistant:\n'
prompt = '실내장식이 뭐야?'
inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=True).to(
    model.device
)
del inputs["token_type_ids"]
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# debug("output")
output = model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=float("inf"),
    pad_token_id=tokenizer.eos_token_id,
    num_return_sequences =1
)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text[0])

## 1.모델 - 학습

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install jsonlines

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [ ]:
# train_ft = pd.read_csv(f'{data_path}train_80000_rule_based.csv')

# 데이터 경로 설정 및 불러오기
datasetName = "train_80000_rule_based.csv"
jsonFileName = "train_80000_rule_based.jsonl"

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = data_path + datasetName
json_file_path = data_path + jsonFileName

In [ ]:
import json

def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기
    df = pd.read_csv(csv_file_path)

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            for q_col in ['question']:
                for a_col in ['answer']:
                    data = {'inputs': row[q_col], 'response': row[a_col]}
                    json.dump(data, json_file, ensure_ascii=False)
                    json_file.write('\n')  # 각 행마다 줄바꿈

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

In [ ]:
import jsonlines
from datasets import Dataset

indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
        indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(data_path)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
['<s>[INST] 면진장치가 뭐야? [/INST] 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. </s>', '<s>[INST] 내진설계의 종류 좀 알려줘 [/INST] 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다. </s>', '<s>[INST] 철골구조의 장점이 뭐야? [/INST] 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다. </s>', '<s>[INST] 철골철근 콘크리트 구조가 뭐야? [/INST] 철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. 철골은 강하고 가벼운 소재로, 높은 내구성과 강도를 가지고 있어 대규모 공간이나 강성이 요구되는 대형 고층건물에 적합합니다. 그러나 화재에 취약하고 녹이 슬면 강성이 떨어지는 단점이 있습니다. 이를 보완하기 위해 철근과 콘크리트를 섞어 사용하는 방식이 개발되었습니다. 철근콘크리트는 철근이 인장력을, 콘크리트가 압축력을 담당하여 강성을 가지며 콘크리트의 열전도가 낮은 재료상 특징을 이용해 서로의 단점을 보완하도록 결합한 구조입니다. 내화성, 내구성, 내진성능이 뛰어나지만 자중이 크고 시공과정이 복잡하며 공사기간이 길고 균일한 시공이 불가능하다는 단점이 있습니다. 철근철골콘크리트는 이 두 가지 구조의 장점을 살려 만든 것으로, 철골 뼈대로 구조를 세우고 그 주변을 철근과 콘크리트로 감싸거나, 철근과 콘크리트로 구조를 만들고 그 위에 철골을 올리는 형태로 만들어집니다. 이렇게 함으로써 철골의 내구성과 강도를 높이면서도 화재에 대한 안전성을 높일 수 있습니다. </s>', '<s>[INST] 철골구조는 어떤 방식이 있어? [/INST] 철골구조는 일반철골구조와 경량철골구조가 있습니다. </s>']


Saving the dataset (0/1 shards):   0%|          | 0/80000 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 80000
})


In [ ]:
indataset

Dataset({
    features: ['text'],
    num_rows: 80000
})

In [ ]:
!pip install peft

In [ ]:
!pip install trl

In [ ]:
!pip install bitsandbytes

In [ ]:
# import locale
# locale.setlocale(locale.LC_ALL, '')

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# Load dataset (you can process it here)
dataset = indataset

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)


In [ ]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and True:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


Your GPU supports bfloat16: accelerate training with bf16=True


In [ ]:
# model.config.use_cache = False
# model.config.pretraining_tp = 1

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
output_dir = f'{data_path}megast_0302'

In [ ]:

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=200,
    logging_steps=16,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False, # a100아닌경우 False
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.05,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)



In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): LlamaForCausalLM(
              (model): LlamaModel(
                (embed_tokens): Embedding(32000, 4096, padding_idx=2)
                (layers): ModuleList(
                  (0-47): 48 x LlamaDecoderLayer(
                    (self_attn): LlamaAttention(
                      (q_proj): lora.Linear4bit(
                        (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.1, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=4096, out_features=64, bias=False)
                        )
                        (lora_B): ModuleDict(
                          (de

In [ ]:
import torch

# Assuming 'model' is your PyTorch model
model_size_mb = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024**2)

print(f"Model size: {model_size_mb:.2f} MB")


Model size: 6149.52 MB


In [ ]:
torch.cuda.empty_cache()
model.config.use_cache = False

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(f'{data_path}models/megast_0302')

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
16,1.089700
32,1.078800
48,1.328600
64,1.121000
80,1.086700
96,1.214000
112,1.103400
128,1.064600
144,1.226200
160,1.074100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
16,1.089700
32,1.078800
48,1.328600
64,1.121000
80,1.086700
96,1.214000
112,1.103400
128,1.064600
144,1.226200
160,1.074100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.